### Step 1: Convert intents.json into a Training Dataset

In [20]:
import json
import random
import pandas as pd

# Load intents.json
with open("intents.json", "r") as f:
    data = json.load(f)

# Create a list of (input, response) pairs
conversations = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        for response in intent["responses"]:
            conversations.append({"text": f"User: {pattern}", "response": f"AI: {response}"})

# Shuffle data to mix different categories
random.shuffle(conversations)

# Convert to DataFrame
df = pd.DataFrame(conversations)

# Split into Train (80%) and Test (20%)
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Save to CSV or JSON for easier processing
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("Dataset structured successfully! Training and Testing sets created.")

Dataset structured successfully! Training and Testing sets created.


In [24]:
print(train_df)

                             text  \
0                       User: yes   
1                       User: Ola   
2        User: I am so burned out   
3               User: I feel down   
4                  User: Bye then   
..                            ...   
523   User: I still feel stressed   
524                  User: family   
525         User: I am so useless   
526  User: Nobody understands me.   
527                    User: yeah   

                                              response  
0                       AI: Can you elaborate on that?  
1             AI: Hi there. How are you feeling today?  
2    AI: I am sorry to hear that. What is the reaso...  
3              AI: Why do you think you feel this way?  
4                                 AI: Have a nice day.  
..                                                 ...  
523  AI: I am sorry to hear that. What is the reaso...  
524                              AI: I see. What else?  
525  AI: i first want to let you know that

### Step 2: Tokenize the Dataset

In [1]:
%pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [38]:
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset

# Load GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = tokenizer(
        [" ".join(pair) for pair in zip(examples["text"], examples["response"])],
        truncation=True,
        padding="max_length",
        max_length=32
    )
    inputs["labels"] = inputs["input_ids"].copy()  # GPT-2 needs labels to compute loss
    return inputs

# Load train and test data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Convert to Hugging Face dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Check tokenized data
print(tokenized_train[0])

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

{'text': 'User: yes', 'response': 'AI: Can you elaborate on that?', 'input_ids': [12982, 25, 3763, 9552, 25, 1680, 345, 15962, 319, 326, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [12982, 25, 3763, 9552, 25, 1680, 345, 15962, 319, 326, 30, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]}


### Step 3: Convert Dataset for Training

In [6]:
%pip install torch

Convert to PyTorch/TensorFlow Dataset

In [39]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import DataLoader

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling since this is causal LM
)

train_dataloader = DataLoader(tokenized_train, batch_size=8, shuffle=True, collate_fn=data_collator)
test_dataloader = DataLoader(tokenized_test, batch_size=8, shuffle=False, collate_fn=data_collator)

Load a GPT-2 Model

In [40]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")


### Step 4: Fine-tune GPT-2

Define Training Arguments

In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-mental-health",  # Save model checkpoints
    evaluation_strategy="epoch",        # Evaluate after each epoch
    save_strategy="epoch",              # Save after each epoch
    logging_dir="./logs",               # Logs for tracking progress
    per_device_train_batch_size=8,      # Adjust batch size based on GPU
    per_device_eval_batch_size=8,
    num_train_epochs=5,                 # Number of epochs (adjust as needed)
    weight_decay=0.01,                   # Regularization
    logging_steps=10,
    save_total_limit=2,                  # Keep only last 2 checkpoints
    push_to_hub=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Create Trainer

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)


<ipython-input-42-ce13606b6f57>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Start Training

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.193200,1.024824
2,0.753200,0.787048
3,0.752000,0.692680
4,0.601800,0.657955
5,0.519200,0.640254


TrainOutput(global_step=330, training_loss=0.8551688555515173, metrics={'train_runtime': 268.1859, 'train_samples_per_second': 9.844, 'train_steps_per_second': 1.23, 'total_flos': 43113185280000.0, 'train_loss': 0.8551688555515173, 'epoch': 5.0})

Save and Load the Model

In [44]:
trainer.save_model("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_model")


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.json',
 'fine_tuned_model/merges.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

Test the Model: Generate responses from your fine-tuned chatbot using

In [47]:
from transformers import pipeline

chat_model = pipeline(
    "text-generation",
    model="fine_tuned_model",
    tokenizer="fine_tuned_model",
    truncation=True  # Add this to avoid warnings
)

input_texts = [
    "User: I feel so alone.",
    "User: Nothing feels right anymore.",
    "User: I'm really stressed out about my job."
]
for text in input_texts:
    output = chat_model(text, max_length=50)
    print(output[0]['generated_text'])

Device set to use cuda:0


User: I feel so alone. AI: Oh ok what brings you today?
User: Nothing feels right anymore. AI: I'm sorry to hear that. I'm trying my best to help
User: I'm really stressed out about my job. AI: Ola, you seem really stressed out. What do you think is behind this?
